In [1]:
import os, cv2, re, shutil
import numpy as np
import pandas as pd

In [2]:
#元画像があるDIR
IMG = '/Users/nakatani/labo/2021_root/2021_Dec/seed-pack2_resized'

In [3]:
fnames = sorted(os.listdir(IMG))
fnames[:3]

['138+P-1_211122.png', '138+P-1_211124.png', '138+P-1_211126.png']

In [4]:
#cv2.imshowで表示したウィンドウをクリックした時に座標を取得する関数
def onMouse(event, x, y, flags, params):
    if event == cv2.EVENT_LBUTTONDOWN:
        list_.append(x)
        list_.append(y)

In [5]:
fname = fnames[0]

In [6]:
#このセルを実行するとウィンドウが表示される
#点を選び終わったらなんでも良いのでキーボードを押す
list_all_imgs = []
list_ = [fname]
img = cv2.imread(os.path.join(IMG,fname))
window_name = fname
img_ = cv2.resize(img, dsize=None, fx=0.5, fy=0.5)
cv2.imshow(window_name, img_)
cv2.setMouseCallback(window_name, onMouse)
cv2.waitKey(0)
cv2.destroyAllWindows()
print(list_) # [file_name, x1, y1, ..., x4, y4]
list_all_imgs.append(list_)

['138+P-1_211122.png', 363, 115, 1033, 114, 349, 810, 1057, 793]


In [11]:
df = pd.DataFrame(list_all_imgs)
df.columns = ['fname', 'x1', 'y1', 'x2', 'y2', 'x3', 'y3', 'x4', 'y4']

In [12]:
df

,fname,x1,y1,x2,y2,x3,y3,x4,y4
0,138+P-1_211122.png,363,115,1033,114,349,810,1057,793


In [14]:
SEG = '' #セグメンテーション画像の格納DIR
DST = '' #変換後の画像の格納DIR

In [15]:
previous = ''
for idx, row in df.iterrows():
    fname = row[0]
    img = cv2.imread(os.path.join(SEG, fname))
    root_name = re.findall('(.+)_', fname)[0] # xxx_日付.pngからxxxだけ抜き出し
    if root_name == previous:
        # half sizeで座標を取ったのでxy座標を2倍
        coord = [[row[1]*2, row[2]*2], [row[3]*2, row[4]*2], [row[5]*2, row[6]*2], [row[7]*2, row[8]*2]]
        M = cv2.getPerspectiveTransform(np.float32(coord), np.float32(base_coord))
        dst = cv2.warpPerspective(img,M,(img.shape[1], img.shape[0]))
        cv2.imwrite(os.path.join(DST, fname), dst)
    else:
        cv2.imwrite(os.path.join(DST, fname), img)
        #変換対象の座標
        base_coord = [[row[1]*2, row[2]*2], [row[3]*2, row[4]*2], [row[5]*2, row[6]*2], [row[7]*2, row[8]*2]]
    previous = root_name

error: OpenCV(4.5.5) /Users/runner/work/opencv-python/opencv-python/opencv/modules/imgcodecs/src/loadsave.cpp:801: error: (-215:Assertion failed) !_img.empty() in function 'imwrite'


## 参考

[Pythonで画像の座標を取得する方法を現役エンジニアが解説【初心者向け】](https://techacademy.jp/magazine/51035)

[pandas.DataFrameのforループ処理（イテレーション）](https://note.nkmk.me/python-pandas-dataframe-for-iteration/)

[画像の幾何学変換](http://opencv.jp/opencv-2.1/cpp/geometric_image_transformations.html)

[Python, OpenCVで幾何変換（アフィン変換・射影変換など）](https://note.nkmk.me/python-opencv-warp-affine-perspective/)